In [1]:
import geopandas as gpd
import pandas as pd

def convert_shapefile_to_csv(shapefile_path, output_path):
    """
    Convert a shapefile to CSV format, extracting ZIP codes and coordinates.

    Parameters:
    shapefile_path (str): Path to the .shp file
    output_path (str): Path where the CSV should be saved
    """
    # Read the shapefile
    gdf = gpd.read_file(shapefile_path)

    # Project to a suitable projected CRS (EPSG:3857 - Web Mercator)
    gdf_projected = gdf.to_crs(epsg=3857)

    # Calculate centroids in projected space
    centroids = gdf_projected.geometry.centroid

    # Convert centroids back to lat/long (EPSG:4326)
    centroids_latlong = centroids.to_crs(epsg=4326)

    # Extract coordinates
    gdf['longitude'] = centroids_latlong.x
    gdf['latitude'] = centroids_latlong.y

    # Convert to regular dataframe, dropping the geometry column
    df = pd.DataFrame(gdf.drop(columns='geometry'))

    # Save to CSV
    df.to_csv(output_path, index=False)

    print(f"Converted shapefile to CSV: {output_path}")
    print(f"Number of records processed: {len(df)}")

#convert_shapefile_to_csv("data", "data/output_run2.csv")

In [5]:
zip_lat_long = pd.read_csv("data/zip_lat_long.csv", dtype="str")

zip_state = pd.read_csv("data/zip_state.csv", dtype="str")

In [9]:
def merge_zipcode_data(df_coords, df_locations):
    """
    Merge two dataframes based on ZIP code matching.

    Parameters:
    df_coords: DataFrame containing ZCTA5CE20, INTPTLAT20, INTPTLON20
    df_locations: DataFrame containing PHYSICAL STATE, PHYSICAL ZIP, PHYSICAL CITY

    Returns:
    DataFrame: Merged dataframe with combined information
    """
    # Create copies to avoid modifying original dataframes
    coords_df = df_coords.copy()
    locations_df = df_locations.copy()

    # Rename columns for clarity
    coords_df = coords_df.rename(columns={
        'ZCTA5CE20': 'ZIP_CODE',
        'INTPTLAT20': 'LATITUDE',
        'INTPTLON20': 'LONGITUDE'
    })

    locations_df = locations_df.rename(columns={
        'PHYSICAL ZIP': 'ZIP_CODE',
        'PHYSICAL STATE': 'STATE',
        'PHYSICAL CITY': 'CITY'
    })

    # Merge dataframes on ZIP code
    merged_df = pd.merge(
        coords_df,
        locations_df,
        on='ZIP_CODE',
        how='inner'  # Only keep matches found in both dataframes
    )

    # Print merge statistics
    print(f"Original coordinates records: {len(df_coords)}")
    print(f"Original locations records: {len(df_locations)}")
    print(f"Merged records: {len(merged_df)}")
    print(f"Unmatched records: {len(df_coords) - len(merged_df)} from coordinates, "
          f"{len(df_locations) - len(merged_df)} from locations")

    return merged_df

# Example usage:

merged_data = merge_zipcode_data(zip_lat_long, zip_state)

merged_data.head()

Original coordinates records: 33791
Original locations records: 44196
Merged records: 42288
Unmatched records: -8497 from coordinates, 1908 from locations


,ZIP_CODE,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,LATITUDE,LONGITUDE,longitude,...,AREA NAME,AREA CODE,DISTRICT NAME,DISTRICT NO,DELIVERY ZIPCODE,LOCALE NAME,PHYSICAL DELV ADDR,CITY,STATE,PHYSICAL ZIP 4
0,35592,35592,B5,G6350,S,298552385,235989,+33.7427261,-088.0973903,-88.09670954934533,...,SOUTHERN,4G,AL-MS,350,35592,VERNON,44924 HIGHWAY 17,VERNON,AL,5630
1,35616,35616,B5,G6350,S,559506992,41870756,+34.7395036,-088.0193814,-88.01890615610482,...,SOUTHERN,4G,AL-MS,350,35616,CHEROKEE,215 MAIN ST,CHEROKEE,AL,7317
2,35621,35621,B5,G6350,S,117838488,409438,+34.3350314,-086.7270557,-86.72392051024694,...,SOUTHERN,4G,AL-MS,350,35621,EVA,4456 HIGHWAY 55 E,EVA,AL,7900
3,35651,35651,B5,G6350,S,104521045,574316,+34.4609087,-087.4801507,-87.48045653962154,...,SOUTHERN,4G,AL-MS,350,35651,MOUNT HOPE,7650 COUNTY ROAD 23,MOUNT HOPE,AL,9793
4,36010,36010,B5,G6350,S,335675180,236811,+31.6598950,-085.8128958,-85.81435735505512,...,SOUTHERN,4G,AL-MS,350,36010,BRUNDIDGE,129 N MAIN ST,BRUNDIDGE,AL,9998


In [10]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, asin

def find_zipcodes_near_nm(df, radius_miles=100):
    """
    Find all ZIP codes within specified radius of any New Mexico ZIP code.

    Parameters:
    df: DataFrame with columns ZIP_CODE, STATE, LATITUDE, LONGITUDE
    radius_miles: Search radius in miles (default 100)

    Returns:
    DataFrame: ZIP codes within radius of any NM ZIP code
    """
    def haversine_distance(lat1, lon1, lat2, lon2):
        """Calculate distance between two lat/lon points in miles"""
        R = 3959.87433  # Earth's radius in miles

        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))

        return R * c

    # Convert lat/lon to float
    df = df.copy()
    df['LATITUDE'] = df['LATITUDE'].astype(float)
    df['LONGITUDE'] = df['LONGITUDE'].astype(float)

    # Get New Mexico ZIP codes
    nm_zips = df[df['STATE'] == 'NM']

    if len(nm_zips) == 0:
        raise ValueError("No New Mexico ZIP codes found in dataset")

    # Initialize empty set for results
    nearby_zips = set()

    # For each NM ZIP code
    for _, nm_row in nm_zips.iterrows():
        # Calculate distances to all other ZIP codes
        distances = df.apply(lambda row: haversine_distance(
            nm_row['LATITUDE'], nm_row['LONGITUDE'],
            row['LATITUDE'], row['LONGITUDE']
        ), axis=1)

        # Add ZIP codes within radius to results
        nearby_zips.update(df[distances <= radius_miles]['ZIP_CODE'].tolist())

    # Get full details for nearby ZIP codes
    result_df = df[df['ZIP_CODE'].isin(nearby_zips)].copy()

    # Add column indicating if ZIP code is in NM
    result_df['IS_NM'] = result_df['STATE'] == 'NM'

    # Sort by ZIP code
    result_df = result_df.sort_values('ZIP_CODE')

    print(f"Found {len(result_df)} ZIP codes within {radius_miles} miles of New Mexico")
    print(f"Breakdown by state:")
    print(result_df['STATE'].value_counts())

    return result_df

# Example usage:
nearby_zips = find_zipcodes_near_nm(merged_data)

Found 859 ZIP codes within 100 miles of New Mexico
Breakdown by state:
STATE
NM    399
TX    222
CO    114
AZ    105
UT      9
OK      7
KS      3
Name: count, dtype: int64


In [11]:
nearby_zips.head()

,ZIP_CODE,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,LATITUDE,LONGITUDE,longitude,...,AREA CODE,DISTRICT NAME,DISTRICT NO,DELIVERY ZIPCODE,LOCALE NAME,PHYSICAL DELV ADDR,CITY,STATE,PHYSICAL ZIP 4,IS_NM
34712,67862,67862,B5,G6350,S,727464933,238832,37.488076,-101.952538,-101.95411309827917,...,4J,KS-MO,630,67862,MANTER,216 N MAIN ST,MANTER,KS,9500,False
6518,67950,67950,B5,G6350,S,469041058,0,37.062774,-101.867684,-101.86769249179139,...,4J,KS-MO,630,67950,ELKHART,336 MORTON ST,ELKHART,KS,9998,False
34754,67953,67953,B5,G6350,S,571963909,230324,37.221665,-101.854850,-101.86084521291797,...,4J,KS-MO,630,67953,RICHFIELD,223 7TH ST,RICHFIELD,KS,9500,False
11333,73933,73933,B5,G6350,S,2489244027,4175764,36.740858,-102.579737,-102.58202508535413,...,4G,AR-OK,730,73933,BOISE CITY,114 N CIMARRON AVE,BOISE CITY,OK,9815,False
35549,73937,73937,B5,G6350,S,553423195,2075488,36.566611,-102.759935,-102.76704703908864,...,4G,AR-OK,730,73937,FELT,101 LINCOLN ST,FELT,OK,9800,False


In [12]:
dropped = nearby_zips[nearby_zips["STATE"] != "NM"]

In [16]:
columns_to_write = ["ZIP_CODE", "CITY", "PHYSICAL DELV ADDR", "STATE"]

dropped.to_csv('output.csv', columns=columns_to_write, index=False)